In [1]:
print("jel")

jel


In [5]:
import psycopg2
import psycopg2.extras as extras
import os

def connect():
    return psycopg2.connect(
        dbname=db_config['database'],
        user=db_config['user'],
        password=db_config['password'],
        host=db_config['host'],
        port=db_config['port']
    )

# method to connect and execute an sql file script for database
def exec_sql_file(path):
    # full_path = os.path.join(os.path.dirname(__file__), f'./{path}')
    current_directory = os.getcwd()
    full_path = os.path.join(current_directory, f'./{path}')
    conn = connect()
    cur = conn.cursor()
    with open(full_path, 'r') as file:
        cur.execute(file.read())
    conn.commit()
    conn.close()

# method to connect and get one record from db; based on provide query
def exec_get_one(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    cur.execute(sql, args)
    one = cur.fetchone()
    conn.close()
    return one

# method to connect and get all records from db; based on provide query
def exec_get_all(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    cur.execute(sql, args)
    # https://www.psycopg.org/docs/cursor.html#cursor.fetchall
    list_of_tuples = cur.fetchall()
    conn.close()
    return list_of_tuples

# method to execute a query; based on provided query
def exec_commit(sql, args={}):
    conn = connect()
    cur = conn.cursor()
    result = cur.execute(sql, args)
    conn.commit()
    conn.close()
    return result

# method to excute a bulk query (i.e., to insert chunk of data togather) based on provided query and input data
def execute_df_values(sql, tuples):
    conn = connect()
    cur = conn.cursor()
    result = extras.execute_values(cur, sql, tuples)
    conn.commit()
    conn.close()
    return result


In [2]:
input_files = ["users", "badges", "tags", "posts", "posttags", "comments", "dummy"]
db_config = {
    'host': 'localhost',
    'database': 'pp1',
    'user': 'postgres',
    'password': 'root',
    'port': '5432'
}

data_directory = 'C:\\Users\\mr2714\\OneDrive - rit.edu\\Python_Projects\\BigData\\pdata\\'

chunk_size = 10

print("hello")

hello


In [6]:
users_query = """ DROP TABLE IF EXISTS users CASCADE;

CREATE TABLE users (
    Id INTEGER PRIMARY KEY,
    UserName VARCHAR(50),
    DisplayName VARCHAR(50),
    RegisterDate DATE,
    PerformanceTier INTEGER, 
    Country VARCHAR(50)
);"""

In [10]:
users_insert_query = """
    INSERT INTO Users (Id, UserName, DisplayName,RegisterDate,PerformanceTier,Country)
    VALUES %s
    """

In [13]:
import pandas as pd
exec_commit(users_query)
# Specify the file path
csv_file = r'C:\Users\Muhammad Raees\OneDrive - rit.edu\Python_Projects\pdata\Users.csv'

# Read the file in chunks of 1000 rows at a time
chunk_size = 1000
chunks = pd.read_csv(csv_file, chunksize=chunk_size)

# Process each chunk (example: printing the first few rows)
c_count = 1
for chunk in chunks:
    # print(chunk)  # or any operation on the chunk
    df_values = list(chunk.itertuples(index=False, name=None))
    # print(chunk)
    execute_df_values(users_insert_query,df_values)
    c_count += 1

    if c_count >= 3:
        break
